In [2]:
### http://deeplearning.net/tutorial/rnnslu.html ###
## https://github.com/dennybritz/rnn-tutorial-rnnlm/blob/master/rnn_theano.py ##
#https://github.com/dennybritz/rnn-tutorial-rnnlm/blob/master/train-theano.py#
#http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/#

import numpy as np
import theano
import theano.tensor as T

Using gpu device 0: GeForce GTX 960


In [20]:
#-------------------------------------------------------#
## Start building the model ##
class RecurrentNet(object):
    def __init__(self,num_h=100,word_dim=8000,bptt_truncate=4):
        self.num_h = num_h
        self.word_dim = word_dim
        self.bptt_truncate = bptt_truncate
        # init network param Wih,Who,Whh
        Wih_val = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim),(num_h,word_dim))
        Whh_val = np.random.uniform(-np.sqrt(1./num_h), np.sqrt(1./num_h),(num_h,num_h))
        Who_val = np.random.uniform(-np.sqrt(1./num_h), np.sqrt(1./num_h),(word_dim,num_h))
        # set value to shared variables
        self.Wih = theano.shared(value=Wih_val.astype(theano.config.floatX),borrow=True,name='Wih')
        self.Whh = theano.shared(value=Whh_val.astype(theano.config.floatX),borrow=True,name='Whh')
        self.Who = theano.shared(value=Who_val.astype(theano.config.floatX),borrow=True,name='Who')
        # setup biases
        self.bh = theano.shared(np.zeros((num_h,1)).astype(theano.config.floatX),borrow=True, name='bh')
        self.bo = theano.shared(np.zeros((word_dim,1)).astype(theano.config.floatX),borrow=True, name='bo')
        # state of hidden layer/network
        state_val = np.zeros(shape=(100,1),dtype=theano.config.floatX)
        self.state = theano.shared(value=state_val,borrow=True,name='state')
        # cost
        self.params = self.Whh + self.Wih + self.Who + self.bh + self.bo
        self.error = ((y - t) ** 2).sum()
        
    
    # forward propagation
    def fstep(self,i_t,h_tm1,Whh,Wih,Who,bh,bo):
        # given input i, calculate current state 
        h_t = T.tanh(T.dot(Whh,h_tm1) + T.dot(Wih,i_t) + bh)
        y_t = T.dot(Who,h_tm1) + bo
        return h_t,y_t
    
    def cost(t,y):
        return T.sum(T.nnet.categorical_crossentropy(t, y))
    
       # def error(t,y):
    #    return ((y - t) ** 2).sum()

In [21]:
x = T.matrix('x')
t = T.vector('t')
#y = T.vector('y')
lr = T.scalar('lr')


rn = RecurrentNet()

h0 = T.matrix()

# theano.scan for running the recurrence loop
[h, y], _ = theano.scan(rn.fstep,
                        sequences=x,
                        outputs_info=[h0, None],
                        non_sequences=[rn.Whh, rn.Wih, rn.Who, rn.bh, rn.bo])

In [26]:
#gradients
error = ((y - t) ** 2).sum()
gparams = T.grad(error, [rn.Whh, rn.Wih, rn.Who, rn.bh, rn.bo])

DisconnectedInputError: grad method was asked to compute the gradient with respect to a variable that is not part of the computational graph of the cost, or is used only by a non-differentiable operator: Elemwise{add,no_inplace}.0

In [13]:
# train function
# SGD.
fn = theano.function([h0, x, t, lr],
                     error,
                     updates={rn.Whh: rn.Whh - lr *gWhh,
                             rn.Wih: rn.Wih - lr * gWih,
                             rn.Who: rn.Who - lr * gWho,
                             rn.bh: rn.bh - lr * gbh,
                             rn.bo: rn.bo - lr * gbo}
                            )

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: UserWarning: The parameter 'updates' of theano.function() expects an OrderedDict, got <type 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (theano.compat.python2x.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
